In [1]:
import os
import cv2
import numpy as np
import pandas as pd

from insightface.app import FaceAnalysis

In [4]:
#configure face analysis
faceapp=FaceAnalysis(name='buffalo_l',
                     root='insightface_model',
                     providers=['CPUExecutionProvider'])
faceapp.prepare(ctx_id=0,det_size=(640,640),det_thresh=0.5)

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_l\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_l\w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


In [5]:
#Extract Facial Features

In [6]:
person_info=[]
listdir=os.listdir(path='images')
for folder_name in listdir:
    roll,name=folder_name.split('-')

    #path of each image in respective folder
    img_files=os.listdir(path=f'images/{folder_name}')
    for file in img_files:
        path=f'./images/{folder_name}/{file}'
        #step-1: read the images
        img_arr=cv2.imread(path)

        #step-2: get the info
        result=faceapp.get(img_arr,max_num=1)#return list
        if len(result)>0:
            #step-3: extract facial embedding
            res=result[0]
            embedding=res['embedding']
            #step-4 :save all info name,role,embedding in a list
            person_info.append([name,roll,embedding])

d:\face_detection_project\attendance_system1\lib\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


In [4]:
dataframe=pd.DataFrame(person_info,columns=['Name','Roll','Facial Features'])
dataframe #store in redis database

,Name,Roll,Facial Features
0,Aadish Jain,1001,"[1.4249021, 0.5167993, 1.1065452, -0.39924377,..."
1,Abhinav Gangil,1002,"[2.5445485, 1.1459275, -0.2571401, -0.9233791,..."
2,Aditi Wekhande,1003,"[-0.275513, 0.6747664, 1.3501925, 1.6961875, -..."
3,Aditi Wekhande,1003,"[-0.028716847, 0.64408815, 0.5351691, 1.272460..."
4,Aditya Kshitiz,1004,"[2.080579, 0.93143797, 0.7777846, -0.6473296, ..."
...,...,...,...
118,Borra Gnana,2029,"[-0.7547005, 0.68525684, 0.27340716, 0.6212193..."
119,Prasad Akanksha,2063,"[-1.0330994, 1.0762423, -0.025145084, 0.598768..."
120,Prasad Akanksha,2063,"[-0.7762105, 0.9452677, -1.2732902, 0.31856424..."
121,Vedant Upadhyay,2081,"[1.2258575, -0.52781576, 0.51487553, 1.1033506..."


In [5]:
img=cv2.imread('test_images/test1.jpg')
cv2.imshow("test",img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
res_test=faceapp.get(img,max_num=0)
for rt in res_test:
    bbox=rt['bbox'].astype(int)
    score=int(rt['det_score']*100)
    embed_test=rt['embedding']

NameError: name 'img' is not defined

In [7]:
len(embed_test)

512

In [8]:
x_list=dataframe['Facial Features'].tolist()
x=np.asarray(x_list)

In [9]:
x.shape

(123, 512)

In [10]:
from sklearn.metrics import pairwise

In [11]:
y=embed_test.reshape(1,512) #1*512

cosine_similar=pairwise.cosine_similarity(x,y)

In [12]:
data_search=dataframe.copy()
data_search['cosine']=cosine_similar

In [13]:
data_search

,Name,Roll,Facial Features,cosine
0,Aadish Jain,1001,"[1.4249021, 0.5167993, 1.1065452, -0.39924377,...",0.134133
1,Abhinav Gangil,1002,"[2.5445485, 1.1459275, -0.2571401, -0.9233791,...",0.184990
2,Aditi Wekhande,1003,"[-0.275513, 0.6747664, 1.3501925, 1.6961875, -...",0.092489
3,Aditi Wekhande,1003,"[-0.028716847, 0.64408815, 0.5351691, 1.272460...",0.076152
4,Aditya Kshitiz,1004,"[2.080579, 0.93143797, 0.7777846, -0.6473296, ...",0.084487
...,...,...,...,...
118,Borra Gnana,2029,"[-0.7547005, 0.68525684, 0.27340716, 0.6212193...",0.056576
119,Prasad Akanksha,2063,"[-1.0330994, 1.0762423, -0.025145084, 0.598768...",0.119297
120,Prasad Akanksha,2063,"[-0.7762105, 0.9452677, -1.2732902, 0.31856424...",0.100708
121,Vedant Upadhyay,2081,"[1.2258575, -0.52781576, 0.51487553, 1.1033506...",0.751585


In [14]:
s_opt=0.5

In [15]:
datafilter=data_search.query(f'cosine>{s_opt}')
datafilter.reset_index(drop=True,inplace=True)

if len(datafilter)>0:
    argmax=datafilter['cosine'].argmax()
    name_cos,roll_cos=datafilter.loc[argmax][['Name','Roll']]
else:
    name_cos='Unknown'
    role_cos='Unknown'


In [16]:
print(name_cos,roll_cos)

Vedant Upadhyay 2081


In [17]:
datafilter

,Name,Roll,Facial Features,cosine
0,Vedant Upadhyay,2081,"[1.2258575, -0.52781576, 0.51487553, 1.1033506...",0.751585
1,Vedant Upadhyay,2081,"[0.15035063, 0.26126587, 0.65102637, 1.3130103...",0.777796


In [18]:
def ml_search_algorithm(dataframe,feature_column,test_vector,name_roll=['Name','Roll'],thresh=0.5):
    #step-1: take the dataframe(collection of data)
    dataframe=dataframe.copy()
    #step-2: Index face embeding from the dataframe
    x_list=dataframe[feature_column].tolist()
    x=np.asarray(x_list)
    #step-3: Calculate cosine similarity
    similar=pairwise.cosine_similarity(x,test_vector.reshape(1,-1))
    similar_arr=np.array(similar).flatten()
    dataframe['cosine']=similar_arr
    #step-4: filter the data
    data_filter=dataframe.query(f'cosine>={thresh}')
    if len(data_filter)>0:
        #step-5: get the person name
        data_filter.reset_index(drop=True,inplace=True)
        argmax=data_filter['cosine'].argmax()
        person_name,person_role=data_filter.loc[argmax][name_roll]
    else:
        person_name='Unknown'
        person_role='Unknown'

    return person_name,person_role
    

In [23]:
test_image=cv2.imread('test_images/test5.jpg')
cv2.imshow('test image',test_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [24]:
#step-1: take the test image and apply to insight face
results=faceapp.get(test_image)
test_copy=test_image.copy()
xyz=[]
#step-2: use for loop and extract each embedding and pass to ml_search_algorithm

for res in results:
    x1,y1,x2,y2=res['bbox'].astype(int)
    embeddings=res['embedding']
    person_name,person_roll=ml_search_algorithm(dataframe,'Facial Features',test_vector=embeddings,name_roll=['Name','Roll'],thresh=0.5)
    cv2.rectangle(test_copy,(x1,y1),(x2,y2),(0,255,0))
    text_gen=person_name
    if person_name=='Unknown':
        cv2.putText(test_copy,text_gen,(x1,y1),cv2.FONT_HERSHEY_DUPLEX,0.35,(0,0,255),1)
    else:
        cv2.putText(test_copy,text_gen,(x1,y1),cv2.FONT_HERSHEY_DUPLEX,0.35,(0,255,0),1)
        xyz.append(text_gen)
cv2.imshow('test image',test_copy)
cv2.waitKey()
cv2.destroyAllWindows()


c:\python 310\lib\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


['Aditya Kshitiz',
 'Aadish Jain',
 'Siddhesh Nitin',
 'Prince Kumar',
 'Aravapalli Pavan Kumar',
 'Aditya Yadav',
 'Aditi Wekhande',
 'Misha Jain',
 'Aditya Mastud',
 'Deepak Yadav',
 'Sai Sanjana',
 'Amit Tiwari',
 'Vineet Verma',
 'Votte Sriyans']

In [27]:
# www=dataframe.copy()
# www
# www = www.drop(columns=['Facial Features'])
# att=[]
# for item in www['Name']:
#     if(item in xyz):
#         att.append(1)
#     else:
#         att.append(0)
# www['att']=att
# www

,Name,Roll,att
0,Aadish Jain,1001,1
1,Abhinav Gangil,1002,0
2,Aditi Wekhande,1003,1
3,Aditi Wekhande,1003,1
4,Aditya Kshitiz,1004,1
...,...,...,...
118,Borra Gnana,2029,0
119,Prasad Akanksha,2063,0
120,Prasad Akanksha,2063,0
121,Vedant Upadhyay,2081,0


In [44]:
#compressing dataframe

In [28]:
dataframe_compress=dataframe.groupby(by=['Name','Roll']).mean()
dataframe_compress.reset_index(inplace=True)
dataframe_compress

,Name,Roll,Facial Features
0,Aadish Jain,1001,"[1.4249021, 0.5167993, 1.1065452, -0.39924377,..."
1,Abhinav Gangil,1002,"[2.5445485, 1.1459275, -0.2571401, -0.9233791,..."
2,Aditi Wekhande,1003,"[-0.15211493, 0.6594273, 0.94268084, 1.4843241..."
3,Aditya Kshitiz,1004,"[1.6784743, 0.84842515, 0.44553462, -0.7783499..."
4,Aditya Mastud,1005,"[-0.4629953, -0.071729705, -0.083244145, -0.42..."
...,...,...,...
75,Vedant Upadhyay,2081,"[0.68810403, -0.13327494, 0.58295095, 1.208180..."
76,Vijit Balsori,1079,"[0.83771884, 1.1717737, 0.025990069, -0.092498..."
77,Vineet Verma,1080,"[1.4692615, 2.0162005, 0.15505144, 0.8998549, ..."
78,Votte Sriyans,1081,"[0.6702863, -0.64987075, -0.3088921, 1.2300224..."


In [29]:
### convert dataframe into array and save into numpy zip format

In [30]:
xvalues=dataframe_compress.values

In [31]:
col_name=np.array(dataframe_compress.columns)
col_name

array(['Name', 'Roll', 'Facial Features'], dtype=object)

In [32]:
np.savez('dataframe_students.npz',xvalues,col_name)

In [33]:
# load the numpy zip
file_np=np.load('dataframe_students.npz',xvalues,col_name)
file_np.files

['arr_0', 'arr_1']